In [ ]:
pip install pyspark geopandas geopy folium kagglehub tqdm ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.5 MB/s eta 0:00:00


In [ ]:
import os
import time
import pickle
from tqdm import tqdm
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import kagglehub

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws, udf
from pyspark.sql.types import StringType, IntegerType, StructType, StructField

# Initialize Spark session
spark = SparkSession.builder.appName("US_Crime_Geocoding").getOrCreate()

# Setup geolocator with a longer timeout
geolocator = Nominatim(user_agent="crime_map_app", timeout=10)

# Load cache if it exists
cache_file = "geocode_cache.pkl"
if os.path.exists(cache_file):
    with open(cache_file, "rb") as f:
        print("Loading cache...")
        geocode_cache = pickle.load(f)
else:
    print("Creating new cache...")
    geocode_cache = {}

not_found_cache_file = "not_found_cache.pkl"
if os.path.exists(not_found_cache_file):
    with open(not_found_cache_file, "rb") as f:
        print("Loading not found cache...")
        not_found_cache = pickle.load(f)
else:
    print("Creating new not found cache...")
    not_found_cache = {}

# Load dataset from Kaggle
path = kagglehub.dataset_download("mrayushagrawal/us-crime-dataset")
csv_file = os.path.join(path, "US_Crime_DataSet.csv")

# Read CSV file using Spark
df = spark.read.csv(csv_file, header=True, inferSchema=True, encoding="latin1")
print(f"Total records after step 1: {df.count()}")
df = df.select("City", "State", "Year") \
       .withColumnRenamed("City", "city") \
       .withColumnRenamed("State", "state") \
       .withColumnRenamed("Year", "year")
print(f"Total records after step 2: {df.count()}")
df = df.filter(col("year").isNotNull())
print(f"Total records after step 3: {df.count()}")

# Group and count
df_grouped = df.groupBy("city", "state", "year").count().withColumnRenamed("count", "number_of_crimes")

print(f"Total records after grouping: {df_grouped.count()}")
df_grouped.show(5, truncate=False)

# Collect the grouped data to driver for geocoding
rows = df_grouped.collect()

# Geocode each city-state
# Geocode each city-state
results = []
for i, row in enumerate(tqdm(rows)):
    city_state = f"{row['city']}, {row['state']}"

    if i % 500 == 0:
        print(f"[Sample Row {i}] Geocoding: {city_state}")

    if city_state in geocode_cache:
        point = geocode_cache[city_state]
    elif city_state in not_found_cache:
        point = None
    else:
        try:
            location = geolocator.geocode({"city": row['city'], "state": row['state'], "country": "USA"})
            if location:
                point = Point(location.longitude, location.latitude)
                geocode_cache[city_state] = point
                # print(f"[Geocoded] {city_state} -> {point}")
            else:
                print(f"[Not Found] {city_state}")
                not_found_cache[city_state] = None
                point = None
        except Exception as e:
            print(f"[Error] {city_state}: {e}")
            time.sleep(1)
            point = None

    if point:
        results.append({
            "city": row["city"],
            "state": row["state"],
            "year": row["year"],
            "number_of_crimes": row["number_of_crimes"],
            "geometry": point
        })

print(f"Total geocoded locations: {len(results)}")
if results:
    print("First 3 geocoded results:")
    for r in results[:3]:
        print(r)

# Save updated geocode caches
with open(cache_file, "wb") as f:
    pickle.dump(geocode_cache, f)

with open(not_found_cache_file, "wb") as f:
    pickle.dump(not_found_cache, f)

# Convert results to a Pandas DataFrame for saving and future use
import pandas as pd
gdf = pd.DataFrame(results)
print(f"DataFrame shape before saving: {gdf.shape}")

# Save to CSV
processed_csv_file = os.path.join("/kaggle/working", "Processed_US_Crime_DataSet.csv")  # For Kaggle
gdf.to_csv(processed_csv_file, index=False)
print(f"Processed data saved to: {processed_csv_file}")


Creating new cache...
Creating new not found cache...
Total records after step 1: 638454
Total records after step 2: 638454
Total records after step 3: 638454
Total records after grouping: 53615
+-----------+----------+----+----------------+
|city       |state     |year|number_of_crimes|
+-----------+----------+----+----------------+
|Escambia   |Alabama   |1980|2               |
|Faulkner   |Arkansas  |1980|2               |
|Boone      |Kentucky  |1980|2               |
|St. Bernard|Louisiana |1980|1               |
|San Juan   |New Mexico|1980|5               |
+-----------+----------+----+----------------+
only showing top 5 rows



  0%|          | 0/53615 [00:00<?, ?it/s]

[Sample Row 0] Geocoding: Escambia, Alabama


  0%|          | 10/53615 [00:09<14:28:35,  1.03it/s]

[Not Found] Hall, Georgia


...

[Sample Row 52500] Geocoding: Wahkiakum, Washington
[Sample Row 53000] Geocoding: Northumberland, Pennsylvania


 99%|█████████▉| 53008/53615 [50:44<00:01, 408.66it/s]

[Not Found] Oliver, North Dakota


100%|██████████| 53615/53615 [50:45<00:00, 17.60it/s] 


[Sample Row 53500] Geocoding: Williams, Ohio
Total geocoded locations: 42693
First 3 geocoded results:
{'city': 'Escambia', 'state': 'Alabama', 'year': 1980, 'number_of_crimes': 2, 'geometry': <POINT (-87.364 31.072)>}
{'city': 'Faulkner', 'state': 'Arkansas', 'year': 1980, 'number_of_crimes': 2, 'geometry': <POINT (-94.086 34.34)>}
{'city': 'Boone', 'state': 'Kentucky', 'year': 1980, 'number_of_crimes': 2, 'geometry': <POINT (-84.317 37.518)>}
DataFrame shape before saving: (42693, 5)
Processed data saved to: /kaggle/working/Processed_US_Crime_DataSet.csv


In [ ]:
import pandas as pd
import numpy as np

# Load the processed crime dataset
csv_path = "Processed_US_Crime_DataSet.csv"  # or your actual path
df = pd.read_csv(csv_path)

# Ensure the number_of_crimes is numeric
df["number_of_crimes"] = pd.to_numeric(df["number_of_crimes"], errors="coerce")

# Drop rows with missing or invalid crime counts
df = df.dropna(subset=["number_of_crimes"])

# Compute global min and max for normalization
min_val = df["number_of_crimes"].min()
max_val = df["number_of_crimes"].max()

# Compute ranking as normalized inverse of number_of_crimes (lower crime → higher rank)
if max_val != min_val:
    df["crime_ranking"] = 1 - ((df["number_of_crimes"] - min_val) / (max_val - min_val))
else:
    df["crime_ranking"] = np.nan  # All values are the same; cannot rank meaningfully

# Preview result
print("Sample ranked data:")
print(df[["city", "state", "year", "number_of_crimes", "crime_ranking"]].head())

# Optionally save it
df.to_csv("Crime_Ranked.csv", index=False)
print("Saved ranked crime dataset to Crime_Ranked.csv")


Sample ranked data:
          city       state  year  number_of_crimes  crime_ranking
0     Escambia     Alabama  1980                 2       0.999521
1     Faulkner    Arkansas  1980                 2       0.999521
2        Boone    Kentucky  1980                 2       0.999521
3  St. Bernard   Louisiana  1980                 1       1.000000
4     San Juan  New Mexico  1980                 5       0.998084
Saved ranked crime dataset to Crime_Ranked.csv


In [ ]:
import pandas as pd
import re

# Load the ranked crime dataset
crime_df = pd.read_csv("Crime_Ranked.csv")

# Step 1: Rename "city" column to "county"
crime_df.rename(columns={"city": "county"}, inplace=True)

# Step 2: Map full state names to abbreviations
state_name_to_abbr = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
    'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC'
}

# Apply mapping to the state column
crime_df["state"] = crime_df["state"].map(state_name_to_abbr).fillna(crime_df["state"])

# Step 3: Remove "county" or "COUNTY" from county column values and strip whitespace
crime_df["county"] = crime_df["county"].str.replace(r"\bcounty\b", "", flags=re.IGNORECASE, regex=True).str.strip()

# Save the updated DataFrame to a new CSV file
crime_df.to_csv("Crime_Ranked_2.csv", index=False)

print("Processed data saved to Crime_Ranked_2.csv")
print(crime_df.head())


Processed data saved to Crime_Ranked_2.csv
        county state  year  number_of_crimes                         geometry  \
0     Escambia    AL  1980                 2   POINT (-87.3636916 31.0721361)   
1     Faulkner    AR  1980                 2   POINT (-94.0863113 34.3403867)   
2        Boone    KY  1980                 2    POINT (-84.316601 37.5175837)   
3  St. Bernard    LA  1980                 1    POINT (-89.858676 29.8671501)   
4     San Juan    NM  1980                 5  POINT (-108.5288693 36.7348233)   

   crime_ranking  
0       0.999521  
1       0.999521  
2       0.999521  
3       1.000000  
4       0.998084  
